# Set

In [1]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
import platform
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
from collections import Counter
from PIL import Image
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load Data

In [2]:
path = "../data/general.csv"
df = pd.read_csv(path, sep = ',', encoding = 'utf-8', header = 0)
df.head()

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,660023,"인생템입니다. 겨울이면 맨날 입술 트고 갈라지고 장난아닌데, 올 겨울은 촉촉하게 보...",쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,5,88,20,20180205,1.0,기능/효과,올 겨울은 촉촉하게 보내고 있어 좋습니다.,6,1
1,660038,발림성 좋습니다. 지속력 좋네요. 건조해진 입술 살짝 발라줬는데 금방 촉촉해져서 좋...,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,5,86,20,20171121,1.0,발림성,발림성 좋습니다.,2,1
2,660044,양이 작아서 금방 쓰겠어요. 보습력도 별로고요. 배송은 빨랐어요. 재구매는 안 할래요.,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,4,48,11,20171110,-1.0,용량/개수,양이 작아서 금방 쓰겠어요.,4,-1
3,660055,추워지면 늘 입술이 갈라져서 사봤는데 촉촉함이 오래 안가요. 양도 너무 작구요.,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,5,44,11,20170911,-1.0,지속력/유지력,촉촉함이 오래 안가요.,3,-1
4,660060,입소문 듣고 처음 구매해 봤는데 배송이 빨라서 좋았어요. 지금 발라보니 번들거리긴 ...,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,4,59,14,20170824,-1.0,흡수력,번들거리긴 합니다.,2,-1


In [3]:
df = df.drop(['Aspect', 'SentimentText', 'SentimentWord', 'SentimentPolarity'], axis=1)
df.head()

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity
0,660023,"인생템입니다. 겨울이면 맨날 입술 트고 갈라지고 장난아닌데, 올 겨울은 촉촉하게 보...",쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,5,88,20,20180205,1.0
1,660038,발림성 좋습니다. 지속력 좋네요. 건조해진 입술 살짝 발라줬는데 금방 촉촉해져서 좋...,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,5,86,20,20171121,1.0
2,660044,양이 작아서 금방 쓰겠어요. 보습력도 별로고요. 배송은 빨랐어요. 재구매는 안 할래요.,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,4,48,11,20171110,-1.0
3,660055,추워지면 늘 입술이 갈라져서 사봤는데 촉촉함이 오래 안가요. 양도 너무 작구요.,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,5,44,11,20170911,-1.0
4,660060,입소문 듣고 처음 구매해 봤는데 배송이 빨라서 좋았어요. 지금 발라보니 번들거리긴 ...,쇼핑몰,화장품,메이크업/뷰티소품,OO 립 크림 1.5g 1개,4,59,14,20170824,-1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158119 entries, 0 to 158118
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Index            158119 non-null  int64  
 1   RawText          158119 non-null  object 
 2   Source           158119 non-null  object 
 3   Domain           158119 non-null  object 
 4   MainCategory     158119 non-null  object 
 5   ProductName      158114 non-null  object 
 6   ReviewScore      158119 non-null  int64  
 7   Syllable         158119 non-null  int64  
 8   Word             158119 non-null  int64  
 9   RDate            158119 non-null  int64  
 10  GeneralPolarity  144408 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 13.3+ MB


In [5]:
# 결측치 삭제
df = df[df['GeneralPolarity'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144408 entries, 0 to 158118
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Index            144408 non-null  int64  
 1   RawText          144408 non-null  object 
 2   Source           144408 non-null  object 
 3   Domain           144408 non-null  object 
 4   MainCategory     144408 non-null  object 
 5   ProductName      144403 non-null  object 
 6   ReviewScore      144408 non-null  int64  
 7   Syllable         144408 non-null  int64  
 8   Word             144408 non-null  int64  
 9   RDate            144408 non-null  int64  
 10  GeneralPolarity  144408 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 13.2+ MB


In [6]:
df['GeneralPolarity'].value_counts().sort_index()

-1.0    22899
 0.0    29723
 1.0    91786
Name: GeneralPolarity, dtype: int64

In [7]:
df['ReviewScore'].value_counts().sort_index()

0         86
1        931
2        462
3       2669
4      11232
5      71561
10       952
20      1238
30       356
40      1187
50       896
60      3315
70      1733
80      6488
90       781
100    40521
Name: ReviewScore, dtype: int64

In [8]:
df['Syllable'].describe()

count    144408.000000
mean         80.057829
std          41.376395
min           8.000000
25%          53.000000
50%          70.000000
75%          96.000000
max         550.000000
Name: Syllable, dtype: float64

In [9]:
df['Word'].describe()

count    144408.000000
mean         18.172650
std           9.791787
min           1.000000
25%          12.000000
50%          16.000000
75%          22.000000
max         146.000000
Name: Word, dtype: float64

# X,Y split

In [10]:
from sklearn.model_selection import train_test_split

y = df['GeneralPolarity']
y

0         1.0
1         1.0
2        -1.0
3        -1.0
4        -1.0
         ... 
158114    0.0
158115    1.0
158116    0.0
158117    1.0
158118    1.0
Name: GeneralPolarity, Length: 144408, dtype: float64

In [11]:
X = df['RawText']
X

0         인생템입니다. 겨울이면 맨날 입술 트고 갈라지고 장난아닌데, 올 겨울은 촉촉하게 보...
1         발림성 좋습니다. 지속력 좋네요. 건조해진 입술 살짝 발라줬는데 금방 촉촉해져서 좋...
2          양이 작아서 금방 쓰겠어요. 보습력도 별로고요. 배송은 빨랐어요. 재구매는 안 할래요.
3              추워지면 늘 입술이 갈라져서 사봤는데 촉촉함이 오래 안가요. 양도 너무 작구요.
4         입소문 듣고 처음 구매해 봤는데 배송이 빨라서 좋았어요. 지금 발라보니 번들거리긴 ...
                                ...                        
158114     핑크색상이 매우 예뻐서 만족해요 사이즈가 너무 아담해서 타자치기 살짝 답답해서 아쉽네요
158115    화이트샀는데 너무 깔끔하고 좋아요 그리고 컴팩트한 사이지가 너무 귀여워서 좋아요 블...
158116             블루투스 페어링이 잘 연결이 안되서 불편해요. 디자인은 깔끔하니 좋아요.
158117               무게가 굉장히 가벼워서 좋구요. 이동하면서 사용할 수 있어 편리해요.
158118    키보드 자판이 둥글게 생겨서 너무 예뻐요 그리고 자판이 패여서 타자칠때 또각또각 감...
Name: RawText, Length: 144408, dtype: object

In [12]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)